# EPCI 10-2019

Ce notebook sert à préparer les données des collectivités entre 20K et 50K habitants.

On va combiner des informations de ces sources :
* INSEE - données de population des communes : https://www.insee.fr/fr/statistiques/2521169#consulter
* ORE - : 
* NosDonnées - latitude et longitude :  http://www.nosdonnees.fr/wiki/index.php/Fichier:EUCircos_Regions_departements_circonscriptions_communes_gps.csv.gz

In [1]:
import pandas as pd  
import numpy as np

# Updating

1. Engadir colter no notebook 2.communes_20_50 + toBDD
2. Engadir colter no notebook 4.consommation_20_50 + toBDD
3. Exportar communes_20_50 (where insee='insee_da_nova_comuna')
4. Importalos datos en communes_20_50 no servidor
5. Exportar communes_20_50, pero sen os campos que non existen en organismes
7. Importalos datos en organismes_20_50 no servidor
8. Exportar e importar no servidor consommation_20_50 

## Données INSEE

In [2]:
# données population des communes
# source : https://www.insee.fr/fr/statistiques/2521169#consulter

pop = pd.read_csv('data/insee/base_cc.csv', encoding='utf-8', sep=',', 
                  low_memory=False, keep_default_na = False,
                 dtype={'CODGEO': str, 'LIBGEO': str, 'REG': str,
                         'DEP': str, 'P15_POP': np.int64, 
                          'P10_POP': np.int64, 'SUPERF': np.int64,
                          'P15_MEN': np.int64,'P15_LOG': np.int64, 'P15_EMPLT_SAL': np.int64,
                          'ETTOT15': np.int64, 'ETAZ15': np.int64, 
                          'ETBE15': np.int64, 'ETFZ15': np.int64, 
                          'ETGU15': np.int64, 'ETGZ15': np.int64, 'ETOQ15': np.int64,
                          'ETTEF115': np.int64, 'ETTEFP1015': np.int64})
pop = pop.rename(columns={'CODGEO': 'insee', 'LIBGEO': 'libelle', 'REG': 'code_region',
                         'DEP': 'code_dept', 'P15_POP': 'population', 
                          'P10_POP': 'population_2010', 'SUPERF': 'superficie',
                          'P15_MEN': 'menages','P15_LOG': 'logements', 'P15_EMPLT_SAL': 'salaries',
                          'ETTOT15': 'ets_actifs', 'ETAZ15': 'ets_agriculture', 
                          'ETBE15': 'ets_industrie', 'ETFZ15': 'ets_construction', 
                          'ETGU15': 'ets_com_serv', 'ETGZ15': 'ets_repar_auto', 'ETOQ15': 'ets_adm',
                          'ETTEF115': 'ets_moins_10', 'ETTEFP1015': 'ets_plus_10'})
pop.head()

,insee,libelle,code_region,code_dept,population,population_2010,superficie,menages,logements,salaries,ets_actifs,ets_agriculture,ets_industrie,ets_construction,ets_com_serv,ets_repar_auto,ets_adm,ets_moins_10,ets_plus_10
0,55039,Beaumont-en-Verdunois,44,55,0,0,8,0,0,0,1,0,0,0,0,0,1,0,0
1,55050,Bezonvaux,44,55,0,0,9,0,0,0,1,0,0,0,0,0,1,0,0
2,55139,Cumières-le-Mort-Homme,44,55,0,0,6,0,1,0,1,0,0,0,0,0,1,1,0
3,55189,Fleury-devant-Douaumont,44,55,0,0,10,0,0,11,2,0,0,0,1,0,1,0,1
4,55239,Haumont-près-Samogneux,44,55,0,0,11,0,0,0,1,0,0,0,0,0,1,0,0


In [3]:
pop.shape

(36735, 19)

## EPCI

Pra engadilas EPCI a T&E, temos que :
1. asocialos organismos de pop as suas EPCI
2. agregalos datos por EPCI

In [23]:
epci = pd.read_csv('data/insee/epci-10-2019.csv', dtype=str)
epci.head()

,codgeo,libgeo,epci,libepci,dep,reg
0,01001,L'Abergement-Clémenciat,200069193,CC de la Dombes,01,84
1,01002,L'Abergement-de-Varey,240100883,CC de la Plaine de l'Ain,01,84
2,01004,Ambérieu-en-Bugey,240100883,CC de la Plaine de l'Ain,01,84
3,01005,Ambérieux-en-Dombes,200042497,CC Dombes Saône Vallée,01,84
4,01006,Ambléon,200040350,CC Bugey Sud,01,84


In [24]:
# pop & epci = pe

pe = pd.merge(pop, epci, how='left', left_on='insee', right_on='codgeo')
pe.head()

,insee,libelle,code_region,code_dept,population,population_2010,superficie,menages,logements,salaries,...,ets_repar_auto,ets_adm,ets_moins_10,ets_plus_10,codgeo,libgeo,epci,libepci,dep,reg
0,55039,Beaumont-en-Verdunois,44,55,0,0,8,0,0,0,...,0,1,0,0,55039,Beaumont-en-Verdunois,200049187,CA du Grand Verdun,55,44
1,55050,Bezonvaux,44,55,0,0,9,0,0,0,...,0,1,0,0,55050,Bezonvaux,200049187,CA du Grand Verdun,55,44
2,55139,Cumières-le-Mort-Homme,44,55,0,0,6,0,1,0,...,0,1,1,0,55139,Cumières-le-Mort-Homme,200049187,CA du Grand Verdun,55,44
3,55189,Fleury-devant-Douaumont,44,55,0,0,10,0,0,11,...,0,1,0,1,55189,Fleury-devant-Douaumont,200049187,CA du Grand Verdun,55,44
4,55239,Haumont-près-Samogneux,44,55,0,0,11,0,0,0,...,0,1,0,0,55239,Haumont-près-Samogneux,200049187,CA du Grand Verdun,55,44


In [33]:
pe.columns

Index(['insee', 'libelle', 'code_region', 'code_dept', 'population',
       'population_2010', 'superficie', 'menages', 'logements', 'salaries',
       'ets_actifs', 'ets_agriculture', 'ets_industrie', 'ets_construction',
       'ets_com_serv', 'ets_repar_auto', 'ets_adm', 'ets_moins_10',
       'ets_plus_10', 'codgeo', 'libgeo', 'epci', 'libepci', 'dep', 'reg'],
      dtype='object')

In [36]:
#pea = pop + epci + agregados  

pea = pe.groupby(
    ['epci', 'libepci', 'code_dept']).agg(
    {'salaries': 'count', 'population': 'sum', 'population_2010': 'sum', 'superficie': 'sum', 'menages': 'sum', 
     'logements': 'sum', 'ets_actifs': 'sum', 'ets_agriculture': 'sum', 'ets_industrie': 'sum', 
     'ets_construction': 'sum', 'ets_com_serv': 'sum', 'ets_repar_auto': 'sum', 'ets_adm': 'sum', 
     'ets_moins_10': 'sum', 'ets_plus_10': 'sum' })
pea= pea.reset_index()
pea.shape

(1244, 18)

In [37]:
pea = pea.drop_duplicates(subset='epci', keep='first')
pea.shape

(1160, 18)

In [38]:
pea.loc[pea['epci']=='200006682']

,epci,libepci,code_dept,salaries,population,population_2010,superficie,menages,logements,ets_actifs,ets_agriculture,ets_industrie,ets_construction,ets_com_serv,ets_repar_auto,ets_adm,ets_moins_10,ets_plus_10
10,200006682,"CA Beaune, Côte et Sud - Communauté Beaune-Cha...",21,48,44890,45635,520,20241,24426,6507,949,311,375,4244,1523,628,1842,376


In [39]:
pea=pea.rename(columns={'salaries':'nb_communes'})
pea.head()

,epci,libepci,code_dept,nb_communes,population,population_2010,superficie,menages,logements,ets_actifs,ets_agriculture,ets_industrie,ets_construction,ets_com_serv,ets_repar_auto,ets_adm,ets_moins_10,ets_plus_10
0,200000172,CC Faucigny-Glières,74,7,26234,24735,131,10668,12593,2193,54,241,216,1351,380,331,526,196
1,200000438,CC du Pays de Pontchâteau Saint-Gildas-des-Bois,44,9,34808,31849,327,13798,15556,2198,222,153,275,1259,409,289,472,131
2,200000545,CC des Portes de Romilly-sur-Seine,10,6,18671,17659,104,8505,9716,1302,56,88,142,802,294,214,356,106
3,200000628,CC Rhône Lez Provence,84,5,24103,24078,150,10048,11816,2141,139,148,333,1285,443,236,542,117
4,200000800,CC Coeur de Sologne,41,6,10655,10642,338,4711,6828,883,77,52,79,574,160,101,248,54


In [40]:
# to string pra pasalo a BDD

pea=pea.astype(str)
pea.head()

,epci,libepci,code_dept,nb_communes,population,population_2010,superficie,menages,logements,ets_actifs,ets_agriculture,ets_industrie,ets_construction,ets_com_serv,ets_repar_auto,ets_adm,ets_moins_10,ets_plus_10
0,200000172,CC Faucigny-Glières,74,7,26234,24735,131,10668,12593,2193,54,241,216,1351,380,331,526,196
1,200000438,CC du Pays de Pontchâteau Saint-Gildas-des-Bois,44,9,34808,31849,327,13798,15556,2198,222,153,275,1259,409,289,472,131
2,200000545,CC des Portes de Romilly-sur-Seine,10,6,18671,17659,104,8505,9716,1302,56,88,142,802,294,214,356,106
3,200000628,CC Rhône Lez Provence,84,5,24103,24078,150,10048,11816,2141,139,148,333,1285,443,236,542,117
4,200000800,CC Coeur de Sologne,41,6,10655,10642,338,4711,6828,883,77,52,79,574,160,101,248,54


In [41]:
pea = pea.fillna('')
pea.shape

(1160, 18)

## to sql

In [56]:
import pymysql.cursors

connection = pymysql.connect( host='localhost',
                              user='root',
                              passwd='123',
                              db='energie',
                              charset='utf8mb4',
                              cursorclass=pymysql.cursors.DictCursor)

cursor = connection.cursor()

In [43]:
# Insertion BDD
# Problèmes avec les types. Pour simplifier, tout est passé en string / varchar 
# Il a été nécessaire de changer les NaN en ''

with connection.cursor() as cursor:
    
    sql = ("INSERT INTO `epci` "
           "( epci, libepci, nb_communes, population, population_2010, "
           "  superficie, menages, logements,`ets_actifs`, `ets_agriculture`,  "
           "  `ets_industrie`, `ets_construction`,`ets_com_serv`, `ets_repar_auto`, `ets_adm`, "  
           "  `ets_moins_10`, `ets_plus_10`, code_dept "
           " )"
           "VALUES ("
           " %s, %s, %s, %s, %s,      %s, %s, %s, %s, %s,     %s, %s, %s, %s, %s,     %s, %s, %s)"
          )

    for i, r in pea.iterrows():
        cursor.execute(sql, 
                       (r['epci'], r['libepci'],  r['nb_communes'], r['population'], r['population_2010'], 
                        r['superficie'], r['menages'], r['logements'], r['ets_actifs'], r['ets_agriculture'], 
                        r['ets_industrie'], r['ets_construction'], r['ets_com_serv'], r['ets_repar_auto'], r['ets_adm'], 
                        r['ets_moins_10'], r['ets_plus_10'], r['code_dept'])
                      )

connection.commit() 

## EPCI ++ 

In [46]:
ed = pd.read_csv('data/opendatasoft/contours-geographiques-des-epci-2019.csv',
                 sep=';', dtype=str)
ed.head()

,Geo Point,Geo Shape,ID,Code EPCI,Nom de l'EPCI,Type,Région,Département,Code Région,Code Departement
0,"48.3603281031,6.65008530612","{""type"": ""Polygon"", ""coordinates"": [[[6.524948...",EPCI00000000000000000010,200005957,CC de la Région de Rambervillers,CC,GRAND EST,VOSGES,44,88
1,"42.5525675978,9.40598069163","{""type"": ""Polygon"", ""coordinates"": [[[9.382191...",EPCI00000000000000000126,200036499,CC de Marana-Golo,CC,CORSE,HAUTE-CORSE,94,2B
2,"44.722319753,0.538394640084","{""type"": ""Polygon"", ""coordinates"": [[[0.454037...",EPCI00000000000000000186,200040889,CC de Portes Sud Périgord,CC,NOUVELLE-AQUITAINE,DORDOGNE,75,24
3,"46.7461512714,5.26526782465","{""type"": ""Polygon"", ""coordinates"": [[[5.130828...",EPCI00000000000000000224,200042414,CC Bresse Revermont 71,CC,BOURGOGNE-FRANCHE-COMTE,SAONE-ET-LOIRE,27,71
4,"42.8450756183,2.12116260511","{""type"": ""Polygon"", ""coordinates"": [[[1.960184...",EPCI00000000000000000253,200043776,CC Pyrénées audoises,CC,OCCITANIE,AUDE,76,11


In [47]:
eplus = pd.merge(pea, ed, how='left', left_on='epci', right_on='Code EPCI')
eplus.shape

(1160, 28)

In [48]:
eplus.columns

Index(['epci', 'libepci', 'code_dept', 'nb_communes', 'population',
       'population_2010', 'superficie', 'menages', 'logements', 'ets_actifs',
       'ets_agriculture', 'ets_industrie', 'ets_construction', 'ets_com_serv',
       'ets_repar_auto', 'ets_adm', 'ets_moins_10', 'ets_plus_10', 'Geo Point',
       'Geo Shape', 'ID', 'Code EPCI', 'Nom de l'EPCI', 'Type', 'Région',
       'Département', 'Code Région', 'Code Departement'],
      dtype='object')

In [49]:
eplus=eplus.drop(['ID', 'Code EPCI', "Nom de l'EPCI", 'Type' ], axis=1)

In [62]:
eplus=eplus.fillna('0')
eplus.isnull().values.any()

False

In [65]:
# Insertion BDD
# Problèmes avec les types. Pour simplifier, tout est passé en string / varchar 
# Il a été nécessaire de changer les NaN en ''

with connection.cursor() as cursor:
    
    sql = ("INSERT INTO `epci` "
           "( epci, libepci, nb_communes, population, population_2010, "
           "  superficie, menages, logements,`ets_actifs`, `ets_agriculture`,  "
           "  `ets_industrie`, `ets_construction`,`ets_com_serv`, `ets_repar_auto`, `ets_adm`, "  
           "  `ets_moins_10`, `ets_plus_10`, code_dept, geo_point, region, " 
           "  departement, code_region, codes_depts )"
           "VALUES ("
           " %s, %s, %s, %s, %s,   %s, %s, %s, %s, %s,    %s, %s, %s, %s, %s,   %s, %s, %s, %s, %s,   %s, %s, %s)"
          )

    for i, r in eplus.iterrows():
        cursor.execute(sql, 
                       (r['epci'], r['libepci'],  r['nb_communes'], r['population'], r['population_2010'], 
                        r['superficie'], r['menages'], r['logements'], r['ets_actifs'], r['ets_agriculture'], 
                        r['ets_industrie'], r['ets_construction'], r['ets_com_serv'], r['ets_repar_auto'], r['ets_adm'], 
                        r['ets_moins_10'], r['ets_plus_10'], r['code_dept'], r['Geo Point'], r['Région'],  
                        r['Département'], r['Code Région'], r['Code Departement'])
                      )

connection.commit() 

In [ ]:
# Copia de un test con insercion de todolos datos (125Mo)

with connection.cursor() as cursor:
    
    sql = ("INSERT INTO `epci` "
           "( epci, libepci, nb_communes, population, population_2010, "
           "  superficie, menages, logements,`ets_actifs`, `ets_agriculture`,  "
           "  `ets_industrie`, `ets_construction`,`ets_com_serv`, `ets_repar_auto`, `ets_adm`, "  
           "  `ets_moins_10`, `ets_plus_10`, code_dept, geo_point, geo_shape, " 
           "  region, departement, code_region, codes_depts )"
           "VALUES ("
           " %s, %s, %s, %s, %s,   %s, %s, %s, %s, %s,    %s, %s, %s, %s, %s,   %s, %s, %s, %s, %s,   %s, %s, %s, %s)"
          )

    for i, r in eplus.iterrows():
        cursor.execute(sql, 
                       (r['epci'], r['libepci'],  r['nb_communes'], r['population'], r['population_2010'], 
                        r['superficie'], r['menages'], r['logements'], r['ets_actifs'], r['ets_agriculture'], 
                        r['ets_industrie'], r['ets_construction'], r['ets_com_serv'], r['ets_repar_auto'], r['ets_adm'], 
                        r['ets_moins_10'], r['ets_plus_10'], r['code_dept'], r['Geo Point'], r['Geo Shape'], 
                        r['Région'], r['Département'], r['Code Région'], r['Code Departement'])
                      )

connection.commit() 

```
ALTER TABLE `epci` CHANGE `id` `id` INT(10) UNSIGNED NOT NULL AUTO_INCREMENT, CHANGE `insee` `epci` VARCHAR(255) CHARACTER SET utf8 COLLATE utf8_general_ci NOT NULL, CHANGE `cp` `libepci` VARCHAR(255) CHARACTER SET utf8 COLLATE utf8_general_ci NULL DEFAULT NULL, CHANGE `libelle` `code_dept` VARCHAR(5) CHARACTER SET utf8 COLLATE utf8_general_ci NOT NULL, CHANGE `EU_circo` `code_region` VARCHAR(5) CHARACTER SET utf8 COLLATE utf8_general_ci NULL DEFAULT NULL, CHANGE `code_region` `nb_communes` VARCHAR(5) CHARACTER SET utf8 COLLATE utf8_general_ci NULL DEFAULT NULL, CHANGE `region` `region` VARCHAR(250) CHARACTER SET utf8 COLLATE utf8_general_ci NULL DEFAULT NULL, CHANGE `code_dept` `dept` VARCHAR(255) CHARACTER SET utf8 COLLATE utf8_general_ci NULL DEFAULT NULL, CHANGE `dept` `population` VARCHAR(10) CHARACTER SET utf8 COLLATE utf8_general_ci NULL DEFAULT NULL, CHANGE `population` `population_2010` VARCHAR(10) CHARACTER SET utf8 COLLATE utf8_general_ci NULL DEFAULT NULL, CHANGE `population_2010` `superficie` VARCHAR(10) CHARACTER SET utf8 COLLATE utf8_general_ci NULL DEFAULT NULL, CHANGE `superficie` `menages` VARCHAR(10) CHARACTER SET utf8 COLLATE utf8_general_ci NULL DEFAULT NULL, CHANGE `menages` `logements` VARCHAR(10) CHARACTER SET utf8 COLLATE utf8_general_ci NULL DEFAULT NULL, CHANGE `logements` `ets_actifs` VARCHAR(10) CHARACTER SET utf8 COLLATE utf8_general_ci NULL DEFAULT NULL, CHANGE `salaries` `ets_agriculture` VARCHAR(10) CHARACTER SET utf8 COLLATE utf8_general_ci NULL DEFAULT NULL, CHANGE `ets_actifs` `ets_industrie` VARCHAR(10) CHARACTER SET utf8 COLLATE utf8_general_ci NULL DEFAULT NULL, CHANGE `ets_agriculture` `ets_construction` VARCHAR(10) CHARACTER SET utf8 COLLATE utf8_general_ci NULL DEFAULT NULL, CHANGE `ets_industrie` `ets_com_serv` VARCHAR(10) CHARACTER SET utf8 COLLATE utf8_general_ci NULL DEFAULT NULL, CHANGE `ets_construction` `ets_repar_auto` VARCHAR(10) CHARACTER SET utf8 COLLATE utf8_general_ci NULL DEFAULT NULL, CHANGE `ets_com_serv` `ets_adm` VARCHAR(10) CHARACTER SET utf8 COLLATE utf8_general_ci NULL DEFAULT NULL, CHANGE `ets_repar_auto` `ets_moins_10` VARCHAR(10) CHARACTER SET utf8 COLLATE utf8_general_ci NULL DEFAULT NULL, CHANGE `ets_adm` VARCHAR(10) CHARACTER SET utf8 COLLATE utf8_general_ci NULL DEFAULT NULL, CHANGE `ets_moins_10` `ets_plus_10` VARCHAR(10) CHARACTER SET utf8 COLLATE utf8_general_ci NULL DEFAULT NULL, CHANGE `ets_plus_10` VARCHAR(10) CHARACTER SET utf8 COLLATE utf8_general_ci NULL DEFAULT NULL, CHANGE `code_epci` VARCHAR(10) CHARACTER SET utf8 COLLATE utf8_general_ci NULL DEFAULT NULL, CHANGE `epci` VARCHAR(250) CHARACTER SET utf8 COLLATE utf8_general_ci NULL DEFAULT NULL, CHANGE `forme` TEXT CHARACTER SET utf8 COLLATE utf8_general_ci NULL DEFAULT NULL, CHANGE `geolocalisation` VARCHAR(250) CHARACTER SET utf8 COLLATE utf8_general_ci NULL DEFAULT NULL, CHANGE `latitude` VARCHAR(250) CHARACTER SET utf8 COLLATE utf8_general_ci NULL DEFAULT NULL, CHANGE `longitude` VARCHAR(250) CHARACTER SET utf8 COLLATE utf8_general_ci NULL DEFAULT NULL; 
```

SELECT `id`, `insee`, `libelle`, `code_region`, `code_dept`, `population`, `population_2010`, `superficie`, `menages`, `logements`, `salaries`, `ets_actifs`, `ets_agriculture`, `ets_industrie`, `ets_construction`, `ets_com_serv`, `ets_repar_auto`, `ets_adm`, `ets_moins_10`, `ets_plus_10`, `code_epci`, `epci`, `forme`, `geolocalisation` FROM `communes_20_50` 

SELECT id, insee, libelle, code_region, code_dept, population, population_2010, superficie, menages, logements, salaries, ets_actifs, ets_agriculture, ets_industrie, ets_construction, ets_com_serv, ets_repar_auto, ets_adm, ets_moins_10, ets_plus_10, code_epci, epci, forme, geolocalisation FROM communes_20_50
 WHERE insee = '44184'
OR insee ='44103'  
OR insee ='44030'
OR insee ='4176'
OR insee ='44052'
OR insee ='44168'
OR insee ='44132'
OR insee ='44210'
OR insee ='44151'
OR insee ='44013'


INSERT INTO `communes_20_50` (`id`, `insee`, `libelle`, `code_region`, `code_dept`, `population`, `population_2010`, `superficie`, `menages`, `logements`, `salaries`, `ets_actifs`, `ets_agriculture`, `ets_industrie`, `ets_construction`, `ets_com_serv`, `ets_repar_auto`, `ets_adm`, `ets_moins_10`, `ets_plus_10`, `code_epci`, `epci`, `forme`, `geolocalisation`) VALUES
(374, '44184', 'Saint-Nazaire', '52', '44', '69784', '67031', '47', '34650', '40160', '33267', '5700', '27', '289', '466', '3917', '1229', '1001', '1364', '437', '244400644', 'CA de la Région Nazairienne et de l\'Estuaire (Carene)', '{\"type\": \"Polygon\", \"coordinates\": [[[-2.296984328, 47.3033160387], [-2.2965113182, 47.3032005093], [-2.2861347342, 47.3092032412], [-2.2798261493, 47.3140056081], [-2.2731875013, 47.3144918084], [-2.2679269851, 47.3143005863], [-2.2630793056, 47.3141788062], [-2.2620383699, 47.314400188], [-2.2578785262, 47.316097451], [-2.2577701366, 47.3163721341], [-2.2568398476000002, 47.3194277716], [-2.2570962029, 47.3200937244], [-2.2547846502, 47.3201858748], [-2.2445629483, 47.3170659395], [-2.2424370041, 47.3152692238], [-2.2395731344, 47.3135355995], [-2.2382568119, 47.3128669903], [-2.2380318595, 47.3125604751], [-2.2352940065, 47.3070191126], [-2.2329160203, 47.3021832278], [-2.2310025457, 47.300854919], [-2.2268129724, 47.2983120105], [-2.2219549145, 47.2954914325], [-2.220844751, 47.2946040482], [-2.2177894204, 47.2920963603], [-2.2175196344, 47.2905359514], [-2.21144303, 47.2878393065], [-2.2074116363, 47.2898014897], [-2.1989041851, 47.2951121675], [-2.1984318178, 47.2955609967], [-2.1949513234999998, 47.2990317105], [-2.1861065246, 47.3056402335], [-2.1847306128, 47.3065958426], [-2.1846014789, 47.306645825], [-2.1844663024, 47.3066058593], [-2.1820028317, 47.3048999186], [-2.1780524567, 47.2964182362], [-2.1727189306, 47.2882787711], [-2.1677625986, 47.279867869], [-2.1812089402, 47.2766365781], [-2.1873608303, 47.2806223617], [-2.1930467074, 47.2773453011], [-2.1954688862, 47.2754711018], [-2.2055602597, 47.2711313969], [-2.2079814658, 47.2708114553], [-2.2105050131, 47.270125973], [-2.2196744325, 47.2674215296], [-2.2212117688, 47.2668199335], [-2.2237390922999998, 47.2654139996], [-2.224422512, 47.2648914632], [-2.2246437955, 47.2643870618], [-2.2245477801, 47.2632645108], [-2.2235106608, 47.2611874447], [-2.2243566464, 47.257910287], [-2.2278604253, 47.2563302611], [-2.2335285504, 47.2560607207], [-2.2425956502, 47.2560611352], [-2.2466914047, 47.2559884314], [-2.2474672239, 47.2557773422], [-2.2481621439, 47.2553894641], [-2.2545201134, 47.2516897359], [-2.2574845317, 47.2499049553], [-2.2603620284, 47.2479652397], [-2.2608790876000002, 47.2462779794], [-2.2610999525, 47.2457735047], [-2.2615984164, 47.2450326536], [-2.2629868294, 47.2430847223], [-2.2644020689, 47.2418573611], [-2.2668623721, 47.2404525143], [-2.2698952747, 47.2395647121], [-2.2806682915, 47.2384674885], [-2.2905337808, 47.2364858771], [-2.2953790707, 47.2343956926], [-2.3016402823, 47.2364000319], [-2.3029706239, 47.2372477105], [-2.3028581884, 47.2374775328], [-2.3019629821, 47.238594975], [-2.2982920016, 47.2450953231], [-2.2980012075, 47.2484191298], [-2.2974613094, 47.2538250744], [-2.2973098844, 47.2551376095], [-2.2967248514, 47.2575491089], [-2.2960002445, 47.2591107165], [-2.2929187148, 47.2594591154], [-2.2916230951, 47.2613201731], [-2.2906637322, 47.2637965435], [-2.2905306213, 47.2665473506], [-2.2898938207, 47.2700113974], [-2.2879817556, 47.2773646956], [-2.2878420705, 47.2780459696], [-2.2880294236, 47.2786690075], [-2.2885921332, 47.2798180784], [-2.2899383661, 47.281611688], [-2.2902729596, 47.281643329], [-2.2935638062, 47.2810382762], [-2.2980176733, 47.2802482172], [-2.3026374242, 47.2799304023], [-2.3054517234, 47.281078292], [-2.3106843559, 47.2826250882], [-2.3106303744, 47.2827629054], [-2.3075493059, 47.2846436877], [-2.304822381, 47.2875466885], [-2.3000108317, 47.2962112511], [-2.2969692644, 47.3023711502], [-2.296984328, 47.3033160387]]]}', '47.2801589313, -2.25106183058'),
(375, '44103', 'Montoir-de-Bretagne', '52', '44', '7058', '6793', '37', '2966', '3157', '9013', '603', '10', '78', '50', '403', '117', '62', '145', '113', '244400644', 'CA de la Région Nazairienne et de l\'Estuaire (Carene)', '{\"type\": \"Polygon\", \"coordinates\": [[[-2.1677625986, 47.279867869], [-2.1727189306, 47.2882787711], [-2.1780524567, 47.2964182362], [-2.1820028317, 47.3048999186], [-2.1844663024, 47.3066058593], [-2.1846014789, 47.306645825], [-2.1846088422, 47.3067357138], [-2.1707671802, 47.3169629361], [-2.170778853, 47.3170977406], [-2.1783669179, 47.3237843062], [-2.1808678508, 47.3259388539], [-2.1853331264, 47.3287823131], [-2.1901775862, 47.331014116], [-2.200897331, 47.3355157507], [-2.2013705066, 47.3356325664], [-2.2013895686, 47.3358563551], [-2.2024194365, 47.3362666717], [-2.2034009995, 47.3369037907], [-2.2048175152, 47.3379742823], [-2.2057096046, 47.3391106983], [-2.2059347983, 47.3399260909], [-2.2060321143, 47.3421167379], [-2.2049360302, 47.3417094192], [-2.200257418, 47.3396854551], [-2.1927025548, 47.3357917955], [-2.1916227804, 47.3355640653], [-2.1812157209, 47.3347555546], [-2.1744931294, 47.3350484444], [-2.1700301064, 47.3360936895], [-2.1651831327, 47.3379488473], [-2.1596566674, 47.3408002147], [-2.1525363226, 47.34596482], [-2.1509782601, 47.3495162704], [-2.1481058141, 47.3543808818], [-2.143811767, 47.3596827966], [-2.1443185563, 47.3602043163], [-2.1497131395, 47.3652208782], [-2.1496542577, 47.365312802], [-2.146591233, 47.3667387199], [-2.1462059342, 47.3668884181], [-2.1444015942, 47.3674988613], [-2.1435490177, 47.3676223505], [-2.1425392345, 47.3674361526], [-2.1423189118, 47.3671743745], [-2.1350460468, 47.358333524], [-2.1262076761, 47.3529308902], [-2.1225526861, 47.3496489251], [-2.1161088777, 47.3423284147], [-2.1121925211, 47.3359020946], [-2.1111317418, 47.3311234212], [-2.111811278, 47.3265467956], [-2.1113494954, 47.3249875377], [-2.1073151102, 47.3163136967], [-2.1080522022, 47.3060592769], [-2.112578388, 47.2945894439], [-2.1194383729, 47.294830627], [-2.1253479883, 47.29325683], [-2.1677625986, 47.279867869]]]}', '47.3191891985, -2.14554523434'),
(376, '44030', 'La Chapelle-des-Marais', '52', '44', '4069', '3772', '18', '1627', '1864', '400', '187', '5', '18', '21', '117', '29', '26', '28', '9', '244400644', 'CA de la Région Nazairienne et de l\'Estuaire (Carene)', '{\"type\": \"Polygon\", \"coordinates\": [[[-2.2767152863, 47.4329378235], [-2.2768552798, 47.4330225885], [-2.2775683673, 47.4335793411], [-2.2803696531, 47.4390998681], [-2.2806312181, 47.4398097444], [-2.2807242915, 47.4408872548], [-2.2810153347, 47.445020544], [-2.2804124074, 47.4472520101], [-2.2750001867, 47.4507122806], [-2.27272605, 47.4520640932], [-2.2614767438, 47.4525566759], [-2.2623043089, 47.4559926145], [-2.2610154852, 47.455682859], [-2.2546967422, 47.4551909731], [-2.2448680368, 47.4563682948], [-2.2347329565, 47.459786895], [-2.221504807, 47.4589121463], [-2.2206218649, 47.4516940046], [-2.2225322795, 47.4453111558], [-2.2227482038, 47.4447168567], [-2.2244572671, 47.4445146467], [-2.2277749458000002, 47.4437086958], [-2.2289979213, 47.4432549252], [-2.2298308732, 47.4429062013], [-2.2303529409, 47.4420301647], [-2.2318122985, 47.4389088492], [-2.231922393, 47.438634145500004], [-2.2236635804000002, 47.4358291575], [-2.214831073, 47.4336763632], [-2.2130830097, 47.4310871297], [-2.2127627914, 47.42889212], [-2.2140275947, 47.4234810707], [-2.2142218357, 47.4234281476], [-2.2242269231, 47.4162313766], [-2.2244181545, 47.4161343637], [-2.2302577587, 47.4149576814], [-2.2360638169, 47.4133996822], [-2.2398180932, 47.4122821954], [-2.2410276308, 47.4124594869], [-2.2465129409, 47.4144952876], [-2.2474078454, 47.4148653593], [-2.2512438019, 47.4170331904], [-2.2545262007, 47.4189074736], [-2.2579259029, 47.4206202846], [-2.2674820739, 47.4251963845], [-2.2690107581, 47.4259913726], [-2.2700186486, 47.4268974794], [-2.2724151089, 47.4292797288], [-2.2731462605, 47.4300612354], [-2.2739522799, 47.43169643], [-2.2748938233000002, 47.4333705779], [-2.2767152863, 47.4329378235]]]}', '47.4367150499, -2.2457741576'),
(378, '44052', 'Donges', '52', '44', '7699', '6748', '49', '3038', '3327', '3426', '448', '25', '42', '59', '271', '94', '51', '100', '47', '244400644', 'CA de la Région Nazairienne et de l\'Estuaire (Carene)', '{\"type\": \"Polygon\", \"coordinates\": [[[-2.1055651626, 47.3860711839], [-2.1054068835, 47.3857624119], [-2.1027596014, 47.3842423978], [-2.0993413805, 47.3830214074], [-2.0958211451, 47.3813546593], [-2.0916971025, 47.3788104088], [-2.0910341001, 47.3780249043], [-2.0888421601, 47.3747751842], [-2.0886567001, 47.3741509536], [-2.0886554221, 47.3725299642], [-2.0888786304, 47.3688270409], [-2.0871283856, 47.3565063794], [-2.0867374169, 47.3558000426], [-2.0861638076, 47.3552812262], [-2.0855542483, 47.3551248186], [-2.0742029313, 47.35492805], [-2.069573189, 47.3550591457], [-2.0572484812, 47.3551233505], [-2.0567517738, 47.355092677], [-2.049024988, 47.3542645718], [-2.0385022265, 47.3512400209], [-2.0313195451, 47.3493263326], [-2.0240051899, 47.3474179225], [-2.0193389809, 47.3462875617], [-2.0152977835, 47.34549443], [-2.012611624, 47.3450548583], [-2.0115323462, 47.3448245571], [-2.0109594972, 47.34349534], [-2.0104315345, 47.3418935702], [-2.01030044, 47.3410873964], [-2.0099582331, 47.3399250136], [-2.0061872624, 47.334530349], [-2.0048197577, 47.3339958182], [-2.0043457197, 47.3330682283], [-2.0039460755, 47.3322267292], [-2.0035731756, 47.3308897582], [-2.0036687094, 47.3296245867], [-2.0038641008, 47.3279509923], [-2.0050122634, 47.3192589834], [-2.0071757207, 47.3140870657], [-2.0106322266, 47.3077406589], [-2.0106708959, 47.3049462316], [-2.0104535325, 47.3001774448], [-2.016422857, 47.3006111579], [-2.0196010507, 47.3018257703], [-2.0278029598, 47.30203773], [-2.0319957688, 47.3017087679], [-2.0443069815, 47.3002517791], [-2.0497368655, 47.2989229421], [-2.0509503489, 47.2981357743], [-2.0734712138, 47.2934725483], [-2.073973992, 47.2942461944], [-2.112578388, 47.2945894439], [-2.1080522022, 47.3060592769], [-2.1073151102, 47.3163136967], [-2.1113494954, 47.3249875377], [-2.111811278, 47.3265467956], [-2.1111317418, 47.3311234212], [-2.1121925211, 47.3359020946], [-2.1161088777, 47.3423284147], [-2.1225526861, 47.3496489251], [-2.1262076761, 47.3529308902], [-2.1350460468, 47.358333524], [-2.1423189118, 47.3671743745], [-2.1425392345, 47.3674361526], [-2.1416320675, 47.3676964631], [-2.1384172781, 47.3704784469], [-2.1374598325, 47.3709213715], [-2.136884658, 47.3711687501], [-2.1364520737, 47.3715460359], [-2.1353557086, 47.37271498], [-2.135062628, 47.3731762992], [-2.1350457875, 47.3737625191], [-2.1352986319, 47.3751947227], [-2.1355037451, 47.3772370644], [-2.1351351769, 47.3795709069], [-2.1348281956, 47.3806634161], [-2.1339890106, 47.3817325502], [-2.1336783955, 47.3819700104], [-2.1297957146, 47.3839215016], [-2.1295238173, 47.3838424019], [-2.1238123546, 47.3805935307], [-2.1210345042, 47.3775027987], [-2.1157096687, 47.3741039062], [-2.1155082525, 47.3740667754], [-2.1132260973, 47.3753256253], [-2.1114290781, 47.3776473881], [-2.1101697603, 47.3800383623], [-2.108340824, 47.3827665054], [-2.105697521, 47.3860662657], [-2.1055651626, 47.3860711839]]]}', '47.3332005289, -2.07208241739'),
(379, '44168', 'Saint-Joachim', '52', '44', '3934', '4046', '86', '1709', '2062', '444', '167', '2', '8', '19', '110', '32', '28', '40', '7', '244400644', 'CA de la Région Nazairienne et de l\'Estuaire (Carene)', '{\"type\": \"Polygon\", \"coordinates\": [[[-2.2767152863, 47.4329378235], [-2.2748938233000002, 47.4333705779], [-2.2739522799, 47.43169643], [-2.2731462605, 47.4300612354], [-2.2724151089, 47.4292797288], [-2.2700186486, 47.4268974794], [-2.2690107581, 47.4259913726], [-2.2674820739, 47.4251963845], [-2.2579259029, 47.4206202846], [-2.2545262007, 47.4189074736], [-2.2512438019, 47.4170331904], [-2.2474078454, 47.4148653593], [-2.2465129409, 47.4144952876], [-2.2410276308, 47.4124594869], [-2.2398180932, 47.4122821954], [-2.2360638169, 47.4133996822], [-2.2302577587, 47.4149576814], [-2.2244181545, 47.4161343637], [-2.2242269231, 47.4162313766], [-2.2142218357, 47.4234281476], [-2.2140275947, 47.4234810707], [-2.2077878063, 47.4219241498], [-2.206547178, 47.421387041], [-2.2007252608, 47.4169304678], [-2.200152935, 47.416457199999996], [-2.1994378399, 47.4158549975], [-2.1975925114, 47.414440011], [-2.1939009978, 47.4116118031], [-2.1864294325, 47.4056874856], [-2.1900719104, 47.3978077935], [-2.1868959037, 47.3931944718], [-2.1862439451, 47.3925397639], [-2.1853381098, 47.3920344195], [-2.184933724, 47.3919604585], [-2.1840777394, 47.3920383731], [-2.1796891827, 47.3927060457], [-2.1793816414, 47.392987701], [-2.1783913865, 47.3946034003], [-2.1734730499, 47.3952908406], [-2.1711403075, 47.3936012974], [-2.1642507245, 47.3882172524], [-2.1640258791, 47.3879106038], [-2.1645613947, 47.387168685], [-2.1716334108, 47.3750899449], [-2.1725824967, 47.3729808384], [-2.1724800245, 47.3717684087], [-2.1716019395, 47.3692346128], [-2.1725167836, 47.3653967002], [-2.1745808747, 47.3629013539], [-2.1859988483, 47.3622743858], [-2.1951707015, 47.3609691917], [-2.2028096848, 47.3556685928], [-2.2085864174, 47.3487738727], [-2.2085431511, 47.3467034992], [-2.2065832043, 47.342365366], [-2.2060321143, 47.3421167379], [-2.2059347983, 47.3399260909], [-2.2057096046, 47.3391106983], [-2.2048175152, 47.3379742823], [-2.2034009995, 47.3369037907], [-2.2024194365, 47.3362666717], [-2.2013895686, 47.3358563551], [-2.2013705066, 47.3356325664], [-2.2016300936, 47.3339997014], [-2.2053551703, 47.3318261152], [-2.2053076811, 47.3297108119], [-2.2025015825, 47.3278834914], [-2.2003161749, 47.3259416683], [-2.1985409725, 47.3241645644], [-2.199428935, 47.3237025309], [-2.2023150335, 47.3241288367], [-2.2070334555, 47.3235155854], [-2.2107552273, 47.3228738498], [-2.2121290341, 47.3222789098], [-2.2133793553, 47.3217788216], [-2.2142914923, 47.3216078918], [-2.2167971323, 47.3214645161], [-2.2181350812, 47.3226015347], [-2.2194925364, 47.3241427502], [-2.2180056712, 47.3270930946], [-2.2203710914, 47.3307397887], [-2.2205737386, 47.330776679], [-2.2295521887, 47.3265026026], [-2.2294593405, 47.3261901147], [-2.2280042184, 47.3246714493], [-2.227297383, 47.3241583811], [-2.2264041291, 47.3237881402], [-2.2259118435, 47.3234476401], [-2.2258338908, 47.32331492], [-2.2388059554, 47.3161794864], [-2.2392106179, 47.3154872434], [-2.2385351812, 47.313801743], [-2.2382568119, 47.3128669903], [-2.2395731344, 47.3135355995], [-2.2424370041, 47.3152692238], [-2.2445629483, 47.3170659395], [-2.2547846502, 47.3201858748], [-2.2570962029, 47.3200937244], [-2.2575732574, 47.3210212413], [-2.2615906464, 47.3255748339], [-2.263746377, 47.3288396244], [-2.2698795272, 47.3331011355], [-2.2730479341, 47.3337808665], [-2.276386855, 47.3339538184], [-2.2766985672, 47.3338981337], [-2.278519302, 47.3341527196], [-2.2799299083, 47.3343664013], [-2.2826755455, 47.3350903869], [-2.2846425193, 47.3367466249], [-2.2854134612, 47.3375253217], [-2.2865462412, 47.3397518685], [-2.2864989518, 47.3435515744], [-2.2861312348, 47.3462831835], [-2.2859597465, 47.3481378887], [-2.2895511565, 47.3490298171], [-2.2898143344, 47.3490196217], [-2.2931685608, 47.3464071003], [-2.2968419218, 47.3429841053], [-2.2978586762, 47.3408083529], [-2.2960425984, 47.3390597123], [-2.2985729804, 47.3361245968], [-2.2997826793, 47.3347881198], [-2.301181272, 47.33410074], [-2.3035159871, 47.3335121919], [-2.307279515, 47.3333607784], [-2.3092280361, 47.3332824207], [-2.3112310906, 47.3345982424], [-2.3125538517, 47.3368429524], [-2.3134156913, 47.3417875953], [-2.3159223361, 47.3443218012], [-2.3165056364, 47.3449293625], [-2.3189562999, 47.3479397692], [-2.3188436117, 47.3481686983], [-2.3163621615, 47.3508827723], [-2.3149873221, 47.3518390074], [-2.3148205682, 47.3522057474], [-2.3149021334, 47.3531484846], [-2.3152916294, 47.3538091425], [-2.3153068068, 47.3539888997], [-2.3141726065, 47.3554312408], [-2.3136647905, 47.3564424743], [-2.3132464574, 47.3592532232], [-2.3157091826, 47.3646956401], [-2.3208765667, 47.3661550721], [-2.318607049, 47.3679346065], [-2.3152413667, 47.3696036952], [-2.3087704376, 47.3718234768], [-2.30638003, 47.3725499307], [-2.3036815474, 47.37355928], [-2.3022202041, 47.3742937093], [-2.3007317785, 47.3754795744], [-2.2982991371, 47.378010454], [-2.2974751698, 47.3792146569], [-2.2972843463, 47.3800786877], [-2.2973656457000002, 47.3810214258], [-2.2975992394, 47.3846449791], [-2.2962168715, 47.3876457317], [-2.2955250568, 47.3888447804], [-2.2932386803, 47.391681001], [-2.293557431, 47.3952140387], [-2.2954286942, 47.3978245785], [-2.2969848315, 47.4007670532], [-2.2965630118, 47.4023191814], [-2.2902569674, 47.4038336483], [-2.2899890823, 47.4037989355], [-2.2864556085, 47.4036251118], [-2.2840060632, 47.4052099145], [-2.2803255381, 47.4079242667], [-2.2788388005, 47.4091548032], [-2.2739548873, 47.4140807881], [-2.2727489494, 47.4154798229], [-2.2719976105, 47.4167715989], [-2.2716239394, 47.417822483], [-2.271416884, 47.4200386681], [-2.271398105, 47.4213459094], [-2.2716538053, 47.4220118604], [-2.2753343286, 47.4307856233], [-2.2762564015, 47.4322350958], [-2.2767152863, 47.4329378235]]]}', '47.3718541085, -2.24329537806'),
(380, '44132', 'Pornichet', '52', '44', '10416', '10451', '13', '5219', '12191', '2151', '1303', '5', '48', '94', '996', '231', '160', '280', '42', '244400644', 'CA de la Région Nazairienne et de l\'Estuaire (Carene)', '{\"type\": \"Polygon\", \"coordinates\": [[[-2.3575644161, 47.2728430053], [-2.3567820402, 47.2737309827], [-2.3540845762, 47.2731645096], [-2.3524332461, 47.2724657274], [-2.3519558201, 47.2723046151], [-2.3383631641, 47.2690266161], [-2.3343814708, 47.2681516437], [-2.3333144373, 47.268059117], [-2.3320645918, 47.2681545556], [-2.3310985033, 47.2684643023], [-2.329488771, 47.2689805087], [-2.3258506475000003, 47.2705238545], [-2.3224693397, 47.2723728852], [-2.3187321659, 47.2746865854], [-2.3147047293, 47.2782090004], [-2.3130441891, 47.2807872124], [-2.3124329536, 47.2813764639], [-2.3108928608, 47.2826552316], [-2.3106843559, 47.2826250882], [-2.3054517234, 47.281078292], [-2.3026374242, 47.2799304023], [-2.2980176733, 47.2802482172], [-2.2935638062, 47.2810382762], [-2.2902729596, 47.281643329], [-2.2899383661, 47.281611688], [-2.2885921332, 47.2798180784], [-2.2880294236, 47.2786690075], [-2.2878420705, 47.2780459696], [-2.2879817556, 47.2773646956], [-2.2898938207, 47.2700113974], [-2.2905306213, 47.2665473506], [-2.2906637322, 47.2637965435], [-2.2916230951, 47.2613201731], [-2.2929187148, 47.2594591154], [-2.2960002445, 47.2591107165], [-2.2967248514, 47.2575491089], [-2.2973098844, 47.2551376095], [-2.2974613094, 47.2538250744], [-2.2980012075, 47.2484191298], [-2.2982920016, 47.2450953231], [-2.3019629821, 47.238594975], [-2.3028581884, 47.2374775328], [-2.3029706239, 47.2372477105], [-2.3040608844, 47.2376092098], [-2.3083312222, 47.2399453282], [-2.3154166807, 47.2434378742], [-2.3225268477, 47.2468224164], [-2.3312874013, 47.2501630663], [-2.3353660172, 47.252160615], [-2.3361388962, 47.2526700005], [-2.3396017708, 47.2551884534], [-2.3413484222, 47.2592630812], [-2.3412558113, 47.2597176093], [-2.3414576386, 47.2605203871], [-2.3420962103000003, 47.2617559836], [-2.3427448815, 47.262360391], [-2.3483493236, 47.2674046602], [-2.349619668, 47.2682997487], [-2.354104908, 47.2711360488], [-2.3552895948, 47.2718085581], [-2.3575644161, 47.2728430053]]]}', '47.2619512895, -2.31411009834'),
(381, '44210', 'Trignac', '52', '44', '7636', '7264', '14', '3218', '3561', '3092', '734', '5', '31', '77', '555', '309', '66', '205', '69', '244400644', 'CA de la Région Nazairienne et de l\'Estuaire (Carene)', '{\"type\": \"Polygon\", \"coordinates\": [[[-2.2013705066, 47.3356325664], [-2.200897331, 47.3355157507], [-2.1901775862, 47.331014116], [-2.1853331264, 47.3287823131], [-2.1808678508, 47.3259388539], [-2.1783669179, 47.3237843062], [-2.170778853, 47.3170977406], [-2.1707671802, 47.3169629361], [-2.1846088422, 47.3067357138], [-2.1846014789, 47.306645825], [-2.1847306128, 47.3065958426], [-2.1861065246, 47.3056402335], [-2.1949513234999998, 47.2990317105], [-2.1984318178, 47.2955609967], [-2.1989041851, 47.2951121675], [-2.2074116363, 47.2898014897], [-2.21144303, 47.2878393065], [-2.2175196344, 47.2905359514], [-2.2177894204, 47.2920963603], [-2.220844751, 47.2946040482], [-2.2219549145, 47.2954914325], [-2.2268129724, 47.2983120105], [-2.2310025457, 47.300854919], [-2.2329160203, 47.3021832278], [-2.2352940065, 47.3070191126], [-2.2380318595, 47.3125604751], [-2.2382568119, 47.3128669903], [-2.2385351812, 47.313801743], [-2.2392106179, 47.3154872434], [-2.2388059554, 47.3161794864], [-2.2258338908, 47.32331492], [-2.2259118435, 47.3234476401], [-2.2264041291, 47.3237881402], [-2.227297383, 47.3241583811], [-2.2280042184, 47.3246714493], [-2.2294593405, 47.3261901147], [-2.2295521887, 47.3265026026], [-2.2205737386, 47.330776679], [-2.2203710914, 47.3307397887], [-2.2180056712, 47.3270930946], [-2.2194925364, 47.3241427502], [-2.2181350812, 47.3226015347], [-2.2167971323, 47.3214645161], [-2.2142914923, 47.3216078918], [-2.2133793553, 47.3217788216], [-2.2121290341, 47.3222789098], [-2.2107552273, 47.3228738498], [-2.2070334555, 47.3235155854], [-2.2023150335, 47.3241288367], [-2.199428935, 47.3237025309], [-2.1985409725, 47.3241645644], [-2.2003161749, 47.3259416683], [-2.2025015825, 47.3278834914], [-2.2053076811, 47.3297108119], [-2.2053551703, 47.3318261152], [-2.2016300936, 47.3339997014], [-2.2013705066, 47.3356325664]]]}', '47.3121514984, -2.20661027054'),
(382, '44151', 'Saint-André-des-Eaux', '52', '44', '6197', '5336', '25', '2386', '2727', '1127', '516', '14', '28', '48', '370', '74', '56', '72', '23', '244400644', 'CA de la Région Nazairienne et de l\'Estuaire (Carene)', '{\"type\": \"Polygon\", \"coordinates\": [[[-2.296984328, 47.3033160387], [-2.2969887637, 47.3033609494], [-2.3001736647, 47.3049459119], [-2.3007757197, 47.305011315], [-2.3021076233, 47.3050929575], [-2.3025296269, 47.3046252244], [-2.3028928967, 47.3038452835], [-2.3051849504, 47.3019538961], [-2.3112784035000002, 47.3008411227], [-2.3120264666, 47.3007337555], [-2.3124748395, 47.3006807229], [-2.3135903152, 47.3008010225], [-2.3140634296, 47.3009173807], [-2.3172489402, 47.3032670014], [-2.3230089939, 47.3051302112], [-2.3318885603, 47.3063029128], [-2.3444683104, 47.3077537558], [-2.3513326406, 47.310111503], [-2.3559151936, 47.3128765867], [-2.3571801702, 47.313320724], [-2.361836058, 47.3146637713], [-2.3631188165, 47.3149265624], [-2.3657127796, 47.3150465493], [-2.3656004497, 47.3152764251], [-2.3604328735, 47.3243175177], [-2.3569596412, 47.329280101], [-2.3551850899, 47.3302084235], [-2.350012659, 47.3316797055], [-2.3428788975, 47.3342219098], [-2.3421804870000003, 47.3345653733], [-2.3392927519, 47.3379267411], [-2.3380071959, 47.3399321454], [-2.3313314599, 47.3450528439], [-2.3233519706, 47.3493171164], [-2.3158550703, 47.3534260316], [-2.3152916294, 47.3538091425], [-2.3149021334, 47.3531484846], [-2.3148205682, 47.3522057474], [-2.3149873221, 47.3518390074], [-2.3163621615, 47.3508827723], [-2.3188436117, 47.3481686983], [-2.3189562999, 47.3479397692], [-2.3165056364, 47.3449293625], [-2.3159223361, 47.3443218012], [-2.3134156913, 47.3417875953], [-2.3125538517, 47.3368429524], [-2.3112310906, 47.3345982424], [-2.3092280361, 47.3332824207], [-2.307279515, 47.3333607784], [-2.3035159871, 47.3335121919], [-2.301181272, 47.33410074], [-2.2997826793, 47.3347881198], [-2.2985729804, 47.3361245968], [-2.2960425984, 47.3390597123], [-2.2978586762, 47.3408083529], [-2.2968419218, 47.3429841053], [-2.2931685608, 47.3464071003], [-2.2898143344, 47.3490196217], [-2.2895511565, 47.3490298171], [-2.2859597465, 47.3481378887], [-2.2861312348, 47.3462831835], [-2.2864989518, 47.3435515744], [-2.2865462412, 47.3397518685], [-2.2854134612, 47.3375253217], [-2.2846425193, 47.3367466249], [-2.2826755455, 47.3350903869], [-2.2799299083, 47.3343664013], [-2.278519302, 47.3341527196], [-2.2766985672, 47.3338981337], [-2.276386855, 47.3339538184], [-2.2730479341, 47.3337808665], [-2.2698795272, 47.3331011355], [-2.263746377, 47.3288396244], [-2.2615906464, 47.3255748339], [-2.2575732574, 47.3210212413], [-2.2570962029, 47.3200937244], [-2.2568398476000002, 47.3194277716], [-2.2577701366, 47.3163721341], [-2.2578785262, 47.316097451], [-2.2620383699, 47.314400188], [-2.2630793056, 47.3141788062], [-2.2679269851, 47.3143005863], [-2.2731875013, 47.3144918084], [-2.2798261493, 47.3140056081], [-2.2861347342, 47.3092032412], [-2.2965113182, 47.3032005093], [-2.296984328, 47.3033160387]]]}', '47.3233837084, -2.31217684154'),
(383, '44013', 'Besné', '52', '44', '2958', '2646', '18', '1113', '1195', '442', '126', '9', '15', '20', '66', '21', '16', '36', '11', '244400644', 'CA de la Région Nazairienne et de l\'Estuaire (Carene)', '{\"type\": \"Polygon\", \"coordinates\": [[[-2.1055651626, 47.3860711839], [-2.1063841785, 47.3879322653], [-2.1063914219, 47.3880221559], [-2.1058474962, 47.3934486933], [-2.1055500035, 47.3954418576], [-2.1048545747, 47.3998385294], [-2.1046289081, 47.4011086617], [-2.1043944072, 47.4014779982], [-2.1040821379, 47.4017145371], [-2.1021325388, 47.4030062855], [-2.0987718935, 47.407898804], [-2.09896198, 47.4081087625], [-2.0989857407, 47.4084279572], [-2.0987212584, 47.4088599353], [-2.0983341094, 47.4093992314], [-2.0963991599, 47.4104304203], [-2.0834103543999998, 47.4100749435], [-2.0826676755, 47.409922519], [-2.0818373038, 47.4095042091], [-2.0813322736, 47.4090274609], [-2.0810790857, 47.408361988], [-2.0809743522, 47.4071045806], [-2.0804478304, 47.4063590521], [-2.0760513896, 47.4013463653], [-2.0743079549, 47.399521002], [-2.0705164951, 47.3953855697], [-2.0692810598999998, 47.3940810014], [-2.0689400381, 47.3939587744], [-2.0573541948, 47.3926435797], [-2.0574722498, 47.3924589295], [-2.0599611345, 47.3904715216], [-2.0639900001, 47.3870304316], [-2.0643528319, 47.3866113852], [-2.0627839668, 47.3852743943], [-2.058436001, 47.3816103128], [-2.0537211147, 47.3791320295], [-2.0535197226, 47.3790947914], [-2.0462361281, 47.3800466594], [-2.0459309528, 47.3803738373], [-2.0416667709, 47.3809406599], [-2.0414397517, 47.3805887975], [-2.0408805805, 47.3802495531], [-2.0362413593, 47.3740175141], [-2.0398120418, 47.3656051808], [-2.040943091, 47.3648416365], [-2.0449634324, 47.3637432452], [-2.0487936586, 47.3627422818], [-2.051274572, 47.3622870352], [-2.0537716603, 47.3612013891], [-2.0587776401, 47.3583988613], [-2.0597097865, 47.3576419801], [-2.0600623716, 47.3570880709], [-2.0601234064, 47.3562302473], [-2.0573217387, 47.3552103646], [-2.0572484812, 47.3551233505], [-2.069573189, 47.3550591457], [-2.0742029313, 47.35492805], [-2.0855542483, 47.3551248186], [-2.0861638076, 47.3552812262], [-2.0867374169, 47.3558000426], [-2.0871283856, 47.3565063794], [-2.0888786304, 47.3688270409], [-2.0886554221, 47.3725299642], [-2.0886567001, 47.3741509536], [-2.0888421601, 47.3747751842], [-2.0910341001, 47.3780249043], [-2.0916971025, 47.3788104088], [-2.0958211451, 47.3813546593], [-2.0993413805, 47.3830214074], [-2.1027596014, 47.3842423978], [-2.1054068835, 47.3857624119], [-2.1055651626, 47.3860711839]]]}', '47.3804743499, -2.0751093173');
